In [1]:
# Assembling RoM and PoP for option chains

# STATUS: Incomplete

import pandas as pd
import datetime
import scipy.stats as st
import numpy as np

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
today = datetime.datetime.now().date()

#*****            Set up the Limits         ****
#_______________________________________________

# Standard Deviation limits
call_probability = 0.97  # for Calls
put_probability = 0.95   # for Puts

call_sd = st.norm.ppf(1-(1-call_probability)/2)
put_sd = st.norm.ppf(1-(1-put_probability)/2)

# Read the pickles
df_ohlc = pd.read_pickle(datapath+'df_ohlc.pkl')
df_options = pd.read_pickle(datapath+'df_nse_options.pkl')

# Convert Expiry column into datetime
df_options.Expiry = pd.to_datetime(df_options.Expiry)

# get the max and min of ohlc for each date
df_ohlc['maxp'] = df_ohlc.loc[:, ['O', 'H', 'L', 'C']].max(1) # Max price for calls
df_ohlc['minp'] = df_ohlc.loc[:, ['O', 'H', 'L', 'C']].min(1) # Min price for puts

# get the days to expiry for the options
df_options['DTE'] = df_options.Expiry - datetime.datetime.now()

In [23]:
# make a dictionary of standard deviation for puts and calls
dict_c_sd = [{s: np.std(df_ohlc[df_ohlc.ibSymbol == s].maxp)} for s in df_ohlc.ibSymbol.unique()]
dict_p_sd = [{s: np.std(df_ohlc[df_ohlc.ibSymbol == s].minp)} for s in df_ohlc.ibSymbol.unique()]

In [9]:
df_options.head()

,nseSymbol,Expiry,Strike,undPrice,cOI,cChnginOI,cVolume,cIV,cLTP,cNetChng,...,pBidPrice,pAskPrice,pAskQty,pNetChng,pLTP,pIV,pVolume,pChnginOI,pOI,DTE
0,ACC,2018-11-29,1200.0,1455.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.00,400.0,NaN,0.05,NaN,NaN,NaN,2000.0,2 days 15:28:29.350893
1,ACC,2018-11-29,1220.0,1455.75,NaN,NaN,NaN,NaN,NaN,NaN,...,0.05,NaN,NaN,NaN,1.25,NaN,NaN,NaN,800.0,2 days 15:28:29.350893
2,ACC,2018-11-29,1240.0,1455.75,NaN,NaN,NaN,NaN,NaN,NaN,...,0.05,3.00,1200.0,NaN,1.75,NaN,NaN,NaN,800.0,2 days 15:28:29.350893
3,ACC,2018-11-29,1260.0,1455.75,400.0,NaN,NaN,NaN,209.60,NaN,...,0.40,0.55,800.0,-0.3,0.50,48.90,2.0,NaN,10400.0,2 days 15:28:29.350893
4,ACC,2018-11-29,1280.0,1455.75,2800.0,NaN,NaN,NaN,99.75,NaN,...,0.40,NaN,NaN,-2.0,0.60,45.51,1.0,NaN,3600.0,2 days 15:28:29.350893
